In [15]:
'''
관련 table을 모두 json형식으로 출력
'''

import os
import json
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec

# 환경 변수 설정
os.environ['PINECONE_API_KEY'] = '97760cf5-e5dd-4273-aa2c-977baa17103c'
api_key = os.getenv('PINECONE_API_KEY')
if not api_key:
    raise ValueError("Pinecone API key is not set. Please ensure the PINECONE_API_KEY environment variable is correct.")

# Pinecone 클라이언트 초기화
pc = Pinecone(api_key=api_key)

# 클라우드 및 리전 설정
cloud = os.getenv('PINECONE_CLOUD', 'aws')
region = os.getenv('PINECONE_REGION', 'us-east-1')
spec = ServerlessSpec(cloud=cloud, region=region)

# 인덱스 이름
index_name = 'example-index'

# 인덱스 객체 얻기
index = pc.Index(index_name)

# 모델 로드
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# 검색 구현
def retrieve_related_tables(query_text, top_k=5):
    query_vector = model.encode(query_text).tolist()
    
    results = index.query(top_k=top_k, vector=query_vector, include_metadata=True)
    
    related_tables = {}
    if results['matches']:
        for match in results['matches']:
            score = match['score']
            metadata = match.get('metadata', {})
            
            sheet_name = metadata.get('sheet')
            table_name = metadata.get('table')
            
            if sheet_name and table_name:
                with open(f'output_tables/{sheet_name}.json', 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    table_data = data.get(table_name, [])

                table_key = f"{sheet_name}_{table_name}"
                related_tables[table_key] = {
                    "sheet": sheet_name,
                    "table": table_name,
                    "data": table_data,
                    "score": score
                }
    
    return related_tables

# 예시 사용
query = "일반휴업의 종류를 알려 줘"
related_tables = retrieve_related_tables(query)
if related_tables:
    # LLM에 입력할 데이터
    print("LLM에 입력할 데이터:")
    print(json.dumps(related_tables, indent=4, ensure_ascii=False))


LLM에 입력할 데이터:
{
    "(1) 업무기준_SOP참고자료": {
        "sheet": "(1) 업무기준",
        "table": "SOP참고자료",
        "data": [
            {
                "metadata": {
                    "구분": "기준관련",
                    "열1": "-",
                    "게시물번호": "지식관리> 업무매뉴얼> NEW업무매뉴얼> SC> 주요업무> '휴업'"
                },
                "text": "기준관련 - 지식관리> 업무매뉴얼> NEW업무매뉴얼> SC> 주요업무> '휴업'"
            },
            {
                "metadata": {
                    "구분": "",
                    "열1": 414653,
                    "게시물번호": "게시판> 업무공유> SC광장> SC업무공유> '[운영지원팀] '23년 설(구정) 휴업 프로세스 진행안내의 건'"
                },
                "text": "414653 게시판> 업무공유> SC광장> SC업무공유> '[운영지원팀] '23년 설(구정) 휴업 프로세스 진행안내의 건'"
            },
            {
                "metadata": {
                    "구분": "",
                    "열1": 411702,
                    "게시물번호": "게시판> 업무공유> SC광장> SC업무공유> '[운영지원팀] '휴업BPM 일부 변경 공유의 건'"
                },
                "text": "411702 게시판> 업무공유> SC광장> SC업무공유> '